In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, BatchNormalization, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

In [2]:
# Load data
train_df = pd.read_csv('digit-recognizer/train.csv')
test_df = pd.read_csv('digit-recognizer/test.csv')

In [3]:
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Shuffle the dataset and convert to numpy array
train_df = train_df.sample(frac = 1)
train_x = train_df.iloc[:,1:].values.reshape(train_df.shape[0], 28, 28, 1) / 255.
train_y = train_df.iloc[:,:1].values

train_y = to_categorical(train_y, num_classes=10)

In [6]:
# Build a model
inputs = Input((28, 28, 1))

x = Conv2D(16, (5,5), padding='same', activation='relu')(inputs)
x = MaxPooling2D((2,2))(x)
x = BatchNormalization()(x)

x = Conv2D(32, (3,3), padding='same', activation='relu')(x)
x = MaxPooling2D((2,2))(x)
x = BatchNormalization()(x)

x = Conv2D(64, (3,3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)
x = Flatten()(x)

outs = Dense(10, activation='softmax')(x)

model = Model(inputs, outs)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.001))

In [8]:
# Train the model
model.fit(train_x,train_y, batch_size=32, epochs=10)

Epoch 1/10
1313/1313 [==============================] - 13s 10ms/step - loss: 0.0205 - accuracy: 0.9942
Epoch 2/10
1313/1313 [==============================] - 13s 10ms/step - loss: 0.0181 - accuracy: 0.9941
Epoch 3/10
1313/1313 [==============================] - 12s 9ms/step - loss: 0.0125 - accuracy: 0.9961
Epoch 4/10
1313/1313 [==============================] - 12s 9ms/step - loss: 0.0127 - accuracy: 0.9960
Epoch 5/10
1313/1313 [==============================] - 12s 9ms/step - loss: 0.0099 - accuracy: 0.9966
Epoch 6/10
1313/1313 [==============================] - 12s 9ms/step - loss: 0.0098 - accuracy: 0.9968
Epoch 7/10
1313/1313 [==============================] - 12s 9ms/step - loss: 0.0084 - accuracy: 0.9974
Epoch 8/10
1313/1313 [==============================] - 12s 9ms/step - loss: 0.0078 - accuracy: 0.9972
Epoch 9/10
1313/1313 [==============================] - 12s 9ms/step - loss: 0.0067 - accuracy: 0.9979
Epoch 10/10
1313/1313 [==============================] - 12s 9ms/step -

In [9]:
# Create prediction array
sub_df = pd.DataFrame(columns=['ImageId','Label'])
to_pred = []
for i in range(test_df.shape[0]):
    values = np.expand_dims(test_df.iloc[i].values.reshape(28,28), axis=-1)
    to_pred.append(values)
pred_arr = np.array(to_pred)
preds = model.predict(pred_arr)

 88/875 [==>...........................] - ETA: 1s

2022-11-26 22:22:55.386150: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


875/875 [==============================] - 2s 2ms/step


In [10]:
# Create submission
image_ids = []
labels = []
for i in range(test_df.shape[0]):
    label = np.argmax(preds[i])
    image_ids.append(i+1)
    labels.append(label)
    
sub_df['ImageId'] = image_ids
sub_df['Label'] = labels

sub_df.to_csv('submission.csv', index=False)